## pipeline

In [145]:
import numpy as np
import os
import pickle
import copy
import edt
import matplotlib.pyplot as plt
import time
import cv2
import pandas as pd
from sklearn.metrics.cluster import adjusted_rand_score
from skimage.metrics import adapted_rand_error

import torch
from torch import from_numpy as from_numpy
from torchsummary import summary

import dgl

from func.run_pipeline_super_vox import segment_super_vox_3_channel, semantic_segment_crop_and_cat_3_channel_output, img_3d_erosion_or_expansion, \
generate_super_vox_by_watershed, get_outlayer_of_a_3d_shape, get_crop_by_pixel_val, Cluster_Super_Vox, assign_boudary_voxels_to_cells_with_watershed, \
delete_too_small_cluster, reassign
from func.run_pipeline import segment, assign_boudary_voxels_to_cells, dbscan_of_seg, semantic_segment_crop_and_cat
from func.cal_accuracy import IOU_and_Dice_Accuracy, VOI
from func.network import VoxResNet, CellSegNet_basic_lite
from func.unet_3d_basic import UNet3D_basic
from func.ultis import save_obj, load_obj
import networkx as nx

In [2]:
HMS_data_dict = load_obj("dataset_info/HMS_dataset_info")
HMS_data_dict_test = HMS_data_dict["test"]
print("Test cases: "+str(HMS_data_dict_test.keys()))
case = "135"
print("for test case "+str(case)+" : "+str(HMS_data_dict_test[case]))

# you may load the image using another path
raw_img=np.load(HMS_data_dict_test[case]["raw"]).astype(float)
hand_seg=np.load(HMS_data_dict_test[case]["ins"]).astype(float)

# np.save('seg_foreground_super_voxel_by_ws_graph.npy', seg_foreground_super_voxel_by_ws)

Test cases: dict_keys(['135', '120', '65', '90'])
for test case 135 : {'raw': 'data/CellSeg_dataset/HMS_processed/raw/135.npy', 'background': 'data/CellSeg_dataset/HMS_processed/segmentation/135/135_background_3d_mask.npy', 'edge': 'data/CellSeg_dataset/HMS_processed/segmentation/135/135_edge_3d_mask.npy', 'edge_foreground': 'data/CellSeg_dataset/HMS_processed/segmentation/135/135_edge_foreground_3d_mask.npy', 'edge_background': 'data/CellSeg_dataset/HMS_processed/segmentation/135/135_edge_background_3d_mask.npy', 'boundary': 'data/CellSeg_dataset/HMS_processed/segmentation/135/135_boundary_3d_mask.npy', 'foreground': 'data/CellSeg_dataset/HMS_processed/segmentation/135/135_foreground_3d_mask.npy', 'ins': 'data/CellSeg_dataset/HMS_processed/segmentation/135/135_ins.npy'}


In [3]:
seg_foreground_super_voxel_by_ws = np.load('seg_foreground_super_voxel_by_ws_graph.npy')

In [4]:
len(np.unique(seg_foreground_super_voxel_by_ws))

1951

In [8]:
# get the size of each super voxel

# unique, counts = np.unique(seg_foreground_super_voxel_by_ws, return_counts=True)
# dict(zip(unique, counts))

In [9]:
class Super_Vox_To_Graph_depr():
    def __init__(self, boundary_extend=2):
        super(Super_Vox_To_Graph, self).__init__
        self.boundary_extend = boundary_extend

        self.UN_PROCESSED = 0
        self.LONELY_POINT = -1
        self.A_LARGE_NUM = 100000000

    def fit(self, input_3d_img, restrict_area_3d=None):
        self.input_3d_img = input_3d_img

        if restrict_area_3d is None:
            self.restrict_area_3d = np.array(input_3d_img==0, dtype=np.int8)
        else:
            self.restrict_area_3d = restrict_area_3d

        unique_vals, unique_val_counts = np.unique(self.input_3d_img, return_counts=True)
        unique_val_counts = unique_val_counts[unique_vals>0]
        unique_vals = unique_vals[unique_vals>0]
        sort_locs = np.argsort(unique_val_counts)[::-1]
        self.unique_vals = unique_vals[sort_locs]

        self.val_labels = dict()
        for unique_val in self.unique_vals:
            self.val_labels[unique_val] = self.UN_PROCESSED

        self.val_outlayer_area = dict()
        for idx, unique_val in enumerate(self.unique_vals):
            # print("get val_outlayer area of all vals: "+str(idx/len(self.unique_vals)))
            self.val_outlayer_area[unique_val] = self.A_LARGE_NUM

        """
        neighborhoods_dict:
        {
            voxel: {
                        neighbor_1: touching area,
                        neighbor_2: touching area
                    }
            ...
        }
        """
        neighborhoods_dict = {}
        for idx, current_val in enumerate(self.unique_vals):
            # print('processing: '+str(idx/len(self.unique_vals))+' pixel val: '+str(current_val))
            if self.val_labels[current_val]!=self.UN_PROCESSED:
                continue
            valid_neighbor_vals = self.regionQuery(current_val)
            neighborhoods_dict[current_val] = valid_neighbor_vals
            # if len(valid_neighbor_vals)>0:
            #     # print('Assign label '+str(current_val)+' to current val\'s neighbors: '+str(valid_neighbor_vals))
            #    self.val_labels[current_val] = current_val
            #    self.growCluster(valid_neighbor_vals, current_val)
            # else:
            #    self.val_labels[current_val] = self.LONELY_POINT

        # self.output_3d_img = self.input_3d_img
        return neighborhoods_dict

    def get_outlayer_area(self, current_val):
        current_crop_img, current_restrict_area = get_crop_by_pixel_val(self.input_3d_img, current_val,
                                                                        boundary_extend=self.boundary_extend,
                                                                        crop_another_3d_img_by_the_way=self.restrict_area_3d)
        current_crop_img_onehot = np.array(current_crop_img==current_val, dtype=np.int8)
        current_crop_img_onehot_outlayer = get_outlayer_of_a_3d_shape(current_crop_img_onehot)

        assert current_crop_img_onehot_outlayer.shape == current_restrict_area.shape

        current_crop_img_onehot_outlayer[current_restrict_area>0]=0
        current_crop_outlayer_area = np.sum(current_crop_img_onehot_outlayer)

        return current_crop_outlayer_area

    def regionQuery(self, current_val):
        current_crop_img, current_restrict_area = get_crop_by_pixel_val(self.input_3d_img, current_val,
                                                                        boundary_extend=self.boundary_extend,
                                                                        crop_another_3d_img_by_the_way=self.restrict_area_3d)

        current_crop_img_onehot = np.array(current_crop_img==current_val, dtype=np.int8)
        current_crop_img_onehot_outlayer = get_outlayer_of_a_3d_shape(current_crop_img_onehot)

        assert current_crop_img_onehot_outlayer.shape == current_restrict_area.shape

        current_crop_img_onehot_outlayer[current_restrict_area>0]=0
        current_crop_outlayer_area = np.sum(current_crop_img_onehot_outlayer)

        neighbor_vals, neighbor_val_counts = np.unique(current_crop_img[current_crop_img_onehot_outlayer>0], return_counts=True)
        neighbor_val_counts = neighbor_val_counts[neighbor_vals>0]
        neighbor_vals = neighbor_vals[neighbor_vals>0]

        print("current_crop_outlayer_area: "+str(current_crop_outlayer_area))

        valid_neighbor_vals = self.neighborCheck(neighbor_vals, neighbor_val_counts, current_crop_outlayer_area)


        print("valid_neighbor_vals: "+str(valid_neighbor_vals))

        return valid_neighbor_vals

    def neighborCheck(self, neighbor_vals, neighbor_val_counts, current_crop_outlayer_area):
        neighbor_val_counts = neighbor_val_counts[neighbor_vals>0]
        neighbor_vals = neighbor_vals[neighbor_vals>0]

        valid_neighbor_vals_dict = {}
        for idx, neighbor_val in enumerate(neighbor_vals):
            print("touching_area: "+str(neighbor_val_counts[idx]), end="\r")
            valid_neighbor_vals_dict[neighbor_val] = neighbor_val_counts[idx]

        # double_checked_valid_neighbor_vals = []
        # for valid_neighbor_val in valid_neighbor_vals_dict.keys():
        #    if self.val_labels[valid_neighbor_val]==self.UN_PROCESSED or \
        #     self.val_labels[valid_neighbor_val]==self.LONELY_POINT:
        #        double_checked_valid_neighbor_vals.append(valid_neighbor_val)

        return valid_neighbor_vals_dict

In [171]:
class Super_Vox_To_Graph():
    def __init__(self, boundary_extend=2):
        super(Super_Vox_To_Graph, self).__init__
        self.boundary_extend = boundary_extend

        self.UN_PROCESSED = 0
        self.LONELY_POINT = -1
        self.A_LARGE_NUM = 100000000

    def get_neighbors_and_touching_area(self, input_3d_img, restrict_area_3d=None):
        """
        Parameters
        ----------
        input_3d_img
        restrict_area_3d

        Returns numpy array with each column representing two super voxels touching
                -> shape: supervoxel_1, neighbor_1, touching_area(between supervoxel_1 and neighbor_1)
                          supervoxel_1, neighbor_2, touching_area(between supervoxel_1 and neighbor_2)
                          ...
        -------

        """
        self.input_3d_img = input_3d_img

        if restrict_area_3d is None:
            self.restrict_area_3d = np.array(input_3d_img==0, dtype=np.int8)
        else:
            self.restrict_area_3d = restrict_area_3d

        unique_vals, unique_val_counts = np.unique(self.input_3d_img, return_counts=True)

        unique_val_counts = unique_val_counts[unique_vals>0]
        unique_vals = unique_vals[unique_vals>0]
        sort_locs = np.argsort(unique_val_counts)[::-1]

        self.unique_vals = unique_vals[sort_locs]

        self.val_labels = dict()
        for unique_val in self.unique_vals:
            self.val_labels[unique_val] = self.UN_PROCESSED

        self.val_outlayer_area = dict()
        for idx, unique_val in enumerate(self.unique_vals):
            # print("get val_outlayer area of all vals: "+str(idx/len(self.unique_vals)))
            self.val_outlayer_area[unique_val] = self.A_LARGE_NUM

        """
        neighborhoods:
        np array:
        supervoxel1, neighbor_1, touching_area
        supervoxel1, neighbor_2, touching_area
        ...
        """
        neighborhoods = []
        for idx, current_val in enumerate(self.unique_vals):
            # print('processing: '+str(idx/len(self.unique_vals))+' pixel val: '+str(current_val))
            #if self.val_labels[current_val]!=self.UN_PROCESSED:
            #    continue
            valid_neighbor_vals = self.regionQuery(current_val)
            if len(valid_neighbor_vals) != 0:
                neighborhoods.append(valid_neighbor_vals)
            # if len(valid_neighbor_vals)>0:
            #     # print('Assign label '+str(current_val)+' to current val\'s neighbors: '+str(valid_neighbor_vals))
            #    self.val_labels[current_val] = current_val
            #    self.growCluster(valid_neighbor_vals, current_val)
            # else:
            #    self.val_labels[current_val] = self.LONELY_POINT

        # self.output_3d_img = self.input_3d_img
        neighborhoods = np.vstack(neighborhoods)

        # remove duplicate combinations
        neighborhoods_sorted_pairs = neighborhoods[:,0:2][:, neighborhoods[:,0:2][0, :].argsort()]


        ind = np.argsort(neighborhoods[:,0:2], axis=1)
        neighborhoods_sorted_pairs = np.take_along_axis(neighborhoods[:,0:2], ind, axis=1)

        _, indices_unique = np.unique(neighborhoods_sorted_pairs, axis=0, return_index=True)

        neighborhoods_deduplicated = neighborhoods[indices_unique]

        return neighborhoods_deduplicated

    def get_outlayer_area(self, current_val):
        current_crop_img, current_restrict_area = get_crop_by_pixel_val(self.input_3d_img, current_val,
                                                                        boundary_extend=self.boundary_extend,
                                                                        crop_another_3d_img_by_the_way=self.restrict_area_3d)
        current_crop_img_onehot = np.array(current_crop_img==current_val, dtype=np.int8)
        current_crop_img_onehot_outlayer = get_outlayer_of_a_3d_shape(current_crop_img_onehot)

        assert current_crop_img_onehot_outlayer.shape == current_restrict_area.shape

        current_crop_img_onehot_outlayer[current_restrict_area>0]=0
        current_crop_outlayer_area = np.sum(current_crop_img_onehot_outlayer)

        return current_crop_outlayer_area

    def regionQuery(self, current_val):
        current_crop_img, current_restrict_area = get_crop_by_pixel_val(self.input_3d_img, current_val,
                                                                        boundary_extend=self.boundary_extend,
                                                                        crop_another_3d_img_by_the_way=self.restrict_area_3d)

        current_crop_img_onehot = np.array(current_crop_img==current_val, dtype=np.int8)
        current_crop_img_onehot_outlayer = get_outlayer_of_a_3d_shape(current_crop_img_onehot)

        assert current_crop_img_onehot_outlayer.shape == current_restrict_area.shape

        current_crop_img_onehot_outlayer[current_restrict_area>0]=0
        current_crop_outlayer_area = np.sum(current_crop_img_onehot_outlayer)

        neighbor_vals, neighbor_val_counts = np.unique(current_crop_img[current_crop_img_onehot_outlayer>0], return_counts=True)
        neighbor_val_counts = neighbor_val_counts[neighbor_vals>0]
        neighbor_vals = neighbor_vals[neighbor_vals>0]

        # print("current_crop_outlayer_area: "+str(current_crop_outlayer_area))

        valid_neighbor_vals = self.neighborCheck(current_val, neighbor_vals, neighbor_val_counts, current_crop_outlayer_area)


        # print("valid_neighbor_vals: "+str(valid_neighbor_vals))

        return valid_neighbor_vals

    def neighborCheck(self, current_val, neighbor_vals, neighbor_val_counts, current_crop_outlayer_area):
        neighbor_val_counts = neighbor_val_counts[neighbor_vals>0]
        neighbor_vals = neighbor_vals[neighbor_vals>0]

        valid_neighbor_vals = np.empty((len(neighbor_vals), 3))
        valid_neighbor_vals[:,0] = current_val
        for idx, neighbor_val in enumerate(neighbor_vals):
            # print("touching_area: "+str(neighbor_val_counts[idx]), end="\r")
            # valid_neighbor_vals_dict[neighbor_val] = neighbor_val_counts[idx]
            valid_neighbor_vals[idx, 1] = neighbor_val
            valid_neighbor_vals[idx, 2] = neighbor_val_counts[idx]

        # double_checked_valid_neighbor_vals = []
        # for valid_neighbor_val in valid_neighbor_vals_dict.keys():
        #    if self.val_labels[valid_neighbor_val]==self.UN_PROCESSED or \
        #     self.val_labels[valid_neighbor_val]==self.LONELY_POINT:
        #        double_checked_valid_neighbor_vals.append(valid_neighbor_val)

        return valid_neighbor_vals

    def add_ground_truth_node_labels(self, input_3d_img, groundtruth_img, neighbors_and_touching_area):
        # add ground truth column to neighbors_and_touching_area matrix
        neighbors_and_touching_area = np.c_[(neighbors_and_touching_area,
                                                np.zeros(len(neighbors_and_touching_area)))]
        unique_values = np.unique(np.append(neighbors_and_touching_area[:,0], neighbors_and_touching_area[:,1]))

        # get the ground truth cell label for each super voxel
        groundtruth_labels = {}
        for idx, value in enumerate(unique_values):
            # get values of groundtruth that overlap with each supervoxel
            overlapping_voxels = groundtruth_img[np.where(input_3d_img == value)]
            # get the most occuring groundtruth voxel label for each supervoxel
            gt_label = np.bincount(overlapping_voxels.astype(int)).argmax()

            groundtruth_labels[value] = gt_label

        # set ground 4th column of neighbors_and_touching_area to 1
        # if both supervoxels have the same groundtruth cell label, 0 otherwise
        for idx, col in enumerate(neighbors_and_touching_area):
            if groundtruth_labels[col[0]] == groundtruth_labels[col[1]]:
                neighbors_and_touching_area[idx, 3] = 1

        return neighbors_and_touching_area


    def get_edges_with_voxel_size(self, neighbors, input_3d_img):
        """

        Parameters
        ----------
        neighbors: numpy array with pair_id, voxel1, voxel2, touching_area

        Returns
        -------
        numpy array with pair_id_1, voxel1(pair1), voxel2(pair1), pair_id_2, voxel(pair2), voxel(pair2), size of shared voxel_x <- pairs that share voxel_x

        """

        unique_vals_all, unique_val_counts = np.unique(input_3d_img, return_counts=True)

        # np array: voxel_id, size
        voxel_sizes = np.transpose(np.stack((unique_vals_all, unique_val_counts)))

        # get the unique values that are really present in nodes
        unique_values = np.unique(np.append(neighbors[:,1], neighbors[:,2]))
        all_combinations = []
        for idx, unique_val in enumerate(unique_values):
            # get the entries (combinations of voxels) that include that voxel
            shared_entries = neighbors[np.where(np.logical_or(neighbors[:,1] == unique_val, neighbors[:,2]==unique_val))][:,0:3]
            if len(shared_entries) > 1:
                # combinations = np.stack(np.meshgrid(shared_entries), -1).reshape(-1, 2)
                combinations = []
                for i in range(0, len(shared_entries)-1):
                    for j in range(i+1,len(shared_entries)):
                        combinations.append(np.hstack((shared_entries[i], shared_entries[j])))
                combinations = np.stack(combinations, axis=0)
                # add voxel size
                new_col = np.ones(len(combinations)) * voxel_sizes[np.where(voxel_sizes[:, 0] == unique_val)][0, 1]
                combinations = np.c_[combinations, new_col]
                all_combinations.append(combinations)

        return np.vstack(all_combinations)

    def build_networkx_graph(self, neighbors, edges_with_voxel_size, with_ground_truth=False):
        G = nx.Graph()

        # add nodes
        G.add_nodes_from(list(neighbors[:,0]))

        # add node attributes
        attributes_dict = {}
        if with_ground_truth:
            for neighbor_pair in neighbors:
                attributes_dict[neighbor_pair[0]] = {"touching_area": neighbor_pair[3],
                                                     "is_same_cell": neighbor_pair[4]}
        else:
            for neighbor_pair in neighbors:
                attributes_dict[neighbor_pair[0]] = {"touching_area": neighbor_pair[3]}
        print(attributes_dict)
        nx.set_node_attributes(G, attributes_dict)

        return G

In [172]:
super_vox_to_graph = Super_Vox_To_Graph()

In [15]:
neighbors = super_vox_to_graph.get_neighbors_and_touching_area(seg_foreground_super_voxel_by_ws)

#seg_foreground_super_voxel_by_ws
#hand_seg
len(neighbors)
for n in neighbors:
    #print(f"n = {n}")
    x = neighbors[np.where(np.logical_and(neighbors[:,0] == n[1], neighbors[:,1] == n[0]))]
    print(f"x = {x}")

In [17]:
neighbors_with_gt = super_vox_to_graph.add_ground_truth_node_labels(seg_foreground_super_voxel_by_ws,
                                                                    hand_seg,
                                                                    neighbors)

In [139]:
# add neighbor ids
neighbors = np.c_[np.arange(len(neighbors)), neighbors]

In [140]:
neighbors

array([[0.000e+00, 1.000e+00, 2.000e+00, 1.600e+01],
       [1.000e+00, 1.000e+00, 5.000e+00, 1.100e+01],
       [2.000e+00, 1.000e+00, 8.000e+00, 2.000e+00],
       ...,
       [1.857e+03, 1.943e+03, 1.936e+03, 4.000e+00],
       [1.858e+03, 1.938e+03, 1.940e+03, 2.000e+00],
       [1.859e+03, 1.943e+03, 1.949e+03, 2.800e+01]])

In [143]:
edges_with_voxel_size = super_vox_to_graph.get_edges_with_voxel_size(neighbors, seg_foreground_super_voxel_by_ws)


In [144]:
edges_with_voxel_size

array([[0.000e+00, 1.000e+00, 2.000e+00, ..., 1.000e+00, 5.000e+00,
        6.900e+01],
       [0.000e+00, 1.000e+00, 2.000e+00, ..., 1.000e+00, 8.000e+00,
        6.900e+01],
       [1.000e+00, 1.000e+00, 5.000e+00, ..., 1.000e+00, 8.000e+00,
        6.900e+01],
       ...,
       [1.853e+03, 1.943e+03, 1.927e+03, ..., 1.943e+03, 1.949e+03,
        4.510e+02],
       [1.857e+03, 1.943e+03, 1.936e+03, ..., 1.943e+03, 1.949e+03,
        4.510e+02],
       [1.830e+03, 1.950e+03, 1.888e+03, ..., 1.950e+03, 1.899e+03,
        9.900e+02]])

In [173]:
graph = super_vox_to_graph.build_networkx_graph(neighbors, edges_with_voxel_size)

{0.0: {'touching_area': 16.0}, 1.0: {'touching_area': 11.0}, 2.0: {'touching_area': 2.0}, 3.0: {'touching_area': 2.0}, 4.0: {'touching_area': 1.0}, 5.0: {'touching_area': 3.0}, 6.0: {'touching_area': 1.0}, 7.0: {'touching_area': 5.0}, 8.0: {'touching_area': 16.0}, 9.0: {'touching_area': 3.0}, 10.0: {'touching_area': 1.0}, 11.0: {'touching_area': 24.0}, 12.0: {'touching_area': 4.0}, 13.0: {'touching_area': 8.0}, 14.0: {'touching_area': 42.0}, 15.0: {'touching_area': 5.0}, 16.0: {'touching_area': 18.0}, 17.0: {'touching_area': 4.0}, 18.0: {'touching_area': 25.0}, 19.0: {'touching_area': 54.0}, 20.0: {'touching_area': 2.0}, 21.0: {'touching_area': 3.0}, 22.0: {'touching_area': 4.0}, 23.0: {'touching_area': 33.0}, 24.0: {'touching_area': 8.0}, 25.0: {'touching_area': 17.0}, 26.0: {'touching_area': 19.0}, 27.0: {'touching_area': 26.0}, 28.0: {'touching_area': 3.0}, 29.0: {'touching_area': 1.0}, 30.0: {'touching_area': 1.0}, 31.0: {'touching_area': 4.0}, 32.0: {'touching_area': 5.0}, 33.0: {

In [174]:
graph.nodes()

NodeView((0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0, 101.0, 102.0, 103.0, 104.0, 105.0, 106.0, 107.0, 108.0, 109.0, 110.0, 111.0, 112.0, 113.0, 114.0, 115.0, 116.0, 117.0, 118.0, 119.0, 120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126.0, 127.0, 128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134.0, 135.0, 136.0, 137.0, 138.0, 139.0, 140.0, 141.0, 142.0, 143.0, 144.0, 145.0, 146.0, 147.0, 148.0, 149.0, 150.0, 151.0, 152.0, 153.0, 154.0, 155.0, 156.0, 1

In [176]:
nx.get_node_attributes(graph, "touching_area")

{0.0: 16.0,
 1.0: 11.0,
 2.0: 2.0,
 3.0: 2.0,
 4.0: 1.0,
 5.0: 3.0,
 6.0: 1.0,
 7.0: 5.0,
 8.0: 16.0,
 9.0: 3.0,
 10.0: 1.0,
 11.0: 24.0,
 12.0: 4.0,
 13.0: 8.0,
 14.0: 42.0,
 15.0: 5.0,
 16.0: 18.0,
 17.0: 4.0,
 18.0: 25.0,
 19.0: 54.0,
 20.0: 2.0,
 21.0: 3.0,
 22.0: 4.0,
 23.0: 33.0,
 24.0: 8.0,
 25.0: 17.0,
 26.0: 19.0,
 27.0: 26.0,
 28.0: 3.0,
 29.0: 1.0,
 30.0: 1.0,
 31.0: 4.0,
 32.0: 5.0,
 33.0: 1.0,
 34.0: 3.0,
 35.0: 3.0,
 36.0: 4.0,
 37.0: 29.0,
 38.0: 33.0,
 39.0: 10.0,
 40.0: 1.0,
 41.0: 11.0,
 42.0: 29.0,
 43.0: 2.0,
 44.0: 7.0,
 45.0: 28.0,
 46.0: 29.0,
 47.0: 5.0,
 48.0: 7.0,
 49.0: 5.0,
 50.0: 13.0,
 51.0: 44.0,
 52.0: 23.0,
 53.0: 10.0,
 54.0: 2.0,
 55.0: 14.0,
 56.0: 26.0,
 57.0: 32.0,
 58.0: 1.0,
 59.0: 26.0,
 60.0: 17.0,
 61.0: 17.0,
 62.0: 11.0,
 63.0: 11.0,
 64.0: 8.0,
 65.0: 28.0,
 66.0: 3.0,
 67.0: 2.0,
 68.0: 25.0,
 69.0: 84.0,
 70.0: 18.0,
 71.0: 30.0,
 72.0: 27.0,
 73.0: 68.0,
 74.0: 42.0,
 75.0: 2.0,
 76.0: 1.0,
 77.0: 2.0,
 78.0: 2.0,
 79.0: 2.0,
 80.0: 1.0,

array([16., 11.,  2., ...,  4.,  2., 28.])

In [177]:
neighbors


array([[0.000e+00, 1.000e+00, 2.000e+00, 1.600e+01],
       [1.000e+00, 1.000e+00, 5.000e+00, 1.100e+01],
       [2.000e+00, 1.000e+00, 8.000e+00, 2.000e+00],
       ...,
       [1.857e+03, 1.943e+03, 1.936e+03, 4.000e+00],
       [1.858e+03, 1.938e+03, 1.940e+03, 2.000e+00],
       [1.859e+03, 1.943e+03, 1.949e+03, 2.800e+01]])